# Used Cars Web Scraping

In [1]:
# Importing Libraries/Dependencies
import selenium
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import time
from bs4 import BeautifulSoup
from selenium import webdriver 
import requests
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

In [2]:
# Connecting to the webdriver
driver = webdriver.Chrome(r"C:\web driver\chromedriver.exe")

### Scraping used car data from www.cardekho.com/

In [3]:
# Getting the webpage of mentioned url
url = "https://www.cardekho.com/used-cars+in+india#usedCarCity"
driver.get(url)
time.sleep(3)

In [4]:
# Fetching urls to navigate location page
cities = driver.find_elements_by_xpath("//ul[@class='listing gsc_col-md-12 gsc_col-lg-12']//li/a")

In [5]:
# Creating empty list and grtting links for locations
city = []
for i in cities:
    city.append(i.get_attribute('href'))

In [6]:
city

['https://www.cardekho.com/used-cars+in+ahmedabad',
 'https://www.cardekho.com/used-cars+in+bangalore',
 'https://www.cardekho.com/used-cars+in+mumbai',
 'https://www.cardekho.com/used-cars+in+jaipur',
 'https://www.cardekho.com/used-cars+in+gurgaon',
 'https://www.cardekho.com/used-cars+in+delhi-ncr',
 'https://www.cardekho.com/used-cars+in+chennai',
 'https://www.cardekho.com/used-cars+in+pune',
 'https://www.cardekho.com/used-cars+in+hyderabad',
 'https://www.cardekho.com/used-cars+in+kolkata',
 'https://www.cardekho.com/used-cars+in+agra',
 'https://www.cardekho.com/used-cars+in+lucknow']

In [7]:
# Creating empty lists for scraping data
Car_brand = []
car_price = []
kms_driven = []
fuel = []
transmission = []
city_url = []
car_urls = []

for i in city:
    driver.get(i)
    time.sleep(3)
    
    for _ in range(150):
        time.sleep(1)
        driver.execute_script("window.scrollBy(0,1000)")
    current_url = driver.current_url    
    car = driver.find_elements_by_xpath("//div[@class='gsc_col-xs-7  carsName']/a")
    for i in car:
        car_urls.append(i.get_attribute('href'))        # saving all the car urls for further information collecting purposes   
        
    # Fetching car brand and location urls of the used cars
    for i in driver.find_elements_by_xpath("//div[@class='gsc_col-xs-7  carsName']/a"):
        Car_brand.append(i.text)
        city_url.append(current_url)
    
    # Fetching price of the cars
    for i in driver.find_elements_by_xpath("//span[@class='amnt ']"):
        car_price.append(i.text)
    
    # Fetching driven kilometers of the used cars
    for i in driver.find_elements_by_xpath("//div[@class='truncate dotlist-2']//div[1]"):
        kms_driven.append(i.text)
    
    # Fetching fuel type of the cars
    for i in driver.find_elements_by_xpath("//div[@class='truncate dotlist-2']//div[2]"):
        fuel.append(i.text)
    
    # Fetching transmission of the cars
    for i in driver.find_elements_by_xpath("//div[@class='truncate dotlist-2']//div[3]"):
        transmission.append(i.text)

In [8]:
# Printing length of car features to confirm
print(len(car_urls),len(Car_brand), len(city_url), len(car_price), len(kms_driven), len(fuel), len(transmission))

10748 10748 10748 10748 10748 10748 10748


We have got same length 10748 in all the columns.

In [9]:
# Creating empty list to scrap the specification of the used cars
seats = []
engine_disp = []
milage = []
color = []
max_power = []
front_brake = []
rear_brake = []
cargo_vol = []
height = []
width = []
length = []
weight = []
insp_score = []
top_speed = []

In [10]:
# for loop to get car urls
for i in car_urls:
    driver.get(i)
    time.sleep(2)
    
    # Clicking on view all specifications button to view more features of the cars
    try:
        view_more = driver.find_element_by_xpath("//*[text() = 'View All Specifications' or text() = 'View More']")
        driver.execute_script("arguments[0].scrollIntoView();", view_more)
        driver.execute_script("arguments[0].click();", view_more)
        time.sleep(2)
    except NoSuchElementException:
        pass
    
    # Fetching engine displacement of the cars
    try:
        eng = driver.find_element_by_xpath("//*[text()='Engine']/following-sibling::div") 
        engine_disp.append(eng.text.replace('CC',''))
    except NoSuchElementException:
        engine_disp.append('-')
    
    # Fetching Mileage of the cars
    try:
        mil = driver.find_element_by_xpath("//*[text()='Mileage']/following-sibling::div")
        milage.append(mil.text)
    except NoSuchElementException:
        milage.append('-')
        
    # Fetching seating capacity of the cars
    try:
        sts = driver.find_element_by_xpath("//*[text()='Seating Capacity']/following-sibling::div")
        seats.append(sts.text)
    except NoSuchElementException:
        seats.append('-')
        
    # Fetching colour of the cars
    try:
        clr = driver.find_element_by_xpath("//*[text()='Color']/following-sibling::div")
        color.append(clr.text)
    except NoSuchElementException:
        color.append('-')
        
    # Fetching Maximum power of the cars
    try:
        pwr = driver.find_element_by_xpath("//*[text()='Max Power']/following-sibling::div")
        max_power.append(pwr.text.replace('bhp',''))
    except NoSuchElementException:
        max_power.append('-')
        
    # Fetching Front brake type of the cars
    try:
        fbrk = driver.find_element_by_xpath("//*[text()='Front Brake Type']/following-sibling::div")
        front_brake.append(fbrk.text)
    except NoSuchElementException:
        front_brake.append('-')
        
    # Fetching rear brake type of the cars
    try:
        rbrk = driver.find_element_by_xpath("//*[text()='Rear Brake Type']/following-sibling::div")
        rear_brake.append(rbrk.text)
    except NoSuchElementException:
        rear_brake.append('-')
        
    # Fetching details of cargo volume of the cars
    try:
        vol = driver.find_element_by_xpath("//*[text()='Cargo Volumn']/following-sibling::div")
        cargo_vol.append(vol.text)
    except NoSuchElementException:
        cargo_vol.append('-')
        
    # Fetching top speed of the cars
    try:
        speed = driver.find_element_by_xpath("//*[text()='Top Speed']/following-sibling::div")
        top_speed.append(speed.text)
    except NoSuchElementException:
        top_speed.append('-')
        
    # Fetching height of the cars
    try:
        hgt = driver.find_element_by_xpath("//*[text()='Height']/following-sibling::div")
        height.append(hgt.text.replace('mm',''))
    except NoSuchElementException:
        height.append('-')
        
    # Fetching width of the cars
    try:
        wth = driver.find_element_by_xpath("//*[text()='Width']/following-sibling::div")
        width.append(wth.text.replace('mm',''))
    except NoSuchElementException:
        width.append('-')
        
    # Fetching length of the cars
    try:
        lth = driver.find_element_by_xpath("//*[text()='Length']/following-sibling::div")
        length.append(lth.text.replace('mm',''))
    except NoSuchElementException:
        length.append('-')
        
    # Fetching gross weight of the cars
    try:
        weigh = driver.find_element_by_xpath("//*[text()='Gross Weight']/following-sibling::div")
        weight.append(weigh.text.replace('kg',''))
    except NoSuchElementException:
        weight.append('-')
        
    # Fetching Inspection score of the cars
    try:
        insp = driver.find_element_by_xpath("//div[@class='inspectionRating right-side gsc_col-xs-5 text-right']/span")
        insp_score.append(insp.text)
    except NoSuchElementException:
        insp_score.append('-')

In [11]:
# Printing lengths of all the features
print(len(seats),len(engine_disp),len(milage),len(color),len(max_power),len(front_brake),len(rear_brake),len(cargo_vol),len(height),len(width),len(length),len(weight),len(insp_score),len(top_speed))

10748 10748 10748 10748 10748 10748 10748 10748 10748 10748 10748 10748 10748 10748


All the features have same length

In [12]:
# Creating a dataframe of scraped data
data = list(zip(Car_brand,fuel,kms_driven,engine_disp,transmission,milage,seats,color,max_power,front_brake,rear_brake,cargo_vol,height,width,length,weight,insp_score,top_speed,city_url,car_price))
df = pd.DataFrame(data, columns = ["Car_Name","Fuel_type","Running_in_kms","Endine_disp", "Gear_transmission","Milage_in_km/ltr","Seating_cap","color","Max_power","front_brake_type","rear_brake_type","cargo_volume","height","width","length","Weight","Insp_score","top_speed","City_url","Car_price"])
df

,Car_Name,Fuel_type,Running_in_kms,Endine_disp,Gear_transmission,Milage_in_km/ltr,Seating_cap,color,Max_power,front_brake_type,rear_brake_type,cargo_volume,height,width,length,Weight,Insp_score,top_speed,City_url,Car_price
0,2019 Maruti Swift,Petrol,"11,241 kms",1197,Manual,21.21 kmpl,5,White,81.80,Disc,Drum,268,1530,1735,3840,1315,,-,https://www.cardekho.com/used-cars+in+ahmedabad,6.19 Lakh
1,2018 Maruti Alto K10,Petrol,"27,994 kms",998,Manual,23.95 kmpl,5,Superior white,67.05,Ventilated Disc,Drum,177,1475,1515,3545,1210,,145 Kmph,https://www.cardekho.com/used-cars+in+ahmedabad,3.45 Lakh
2,2016 Maruti Swift,Diesel,"80,728 kms",1248,Manual,25.2 kmpl,5,Silver,74,Ventilated Disc,Drum,204-liters,1530,1735,3840,1405,,155 Kmph,https://www.cardekho.com/used-cars+in+ahmedabad,4.80 Lakh
3,2015 Maruti SX4 S Cross,Diesel,"46,390 kms",1598,Manual,22.7 kmpl,5,Urban Blue,118,Ventilated Disc,Solid Disc,353-litres,1590,1765,4300,1740,,180 Kmph,https://www.cardekho.com/used-cars+in+ahmedabad,6.35 Lakh
4,2019 Jeep Compass,Petrol,"32,294 kms",1368,Automatic,16 kmpl,5,Grey,160.77,Disc,Discs,408-litres,1640,1818,4395,-,,186.69 kmph,https://www.cardekho.com/used-cars+in+ahmedabad,17.05 Lakh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10743,2010 Maruti 800,Petrol,"70,000 kms",796,Manual,16.1 kmpl,4,White,37,Disc,Drum,-,1405,1440,3335,1000,-,144km/hr,https://www.cardekho.com/used-cars+in+lucknow,67500
10744,2012 Tata Indica,Diesel,"32,000 kms",1405,Manual,19.4 kmpl,5,White,70,Disc,Drum,232-litre,1550,1695,3795,-,-,-,https://www.cardekho.com/used-cars+in+lucknow,1.50 Lakh
10745,2005 Hyundai Santro,Petrol,"50,000 kms",999,Manual,-,5,Black,-,-,-,-,-,-,-,-,-,-,https://www.cardekho.com/used-cars+in+lucknow,65000
10746,2017 Hyundai Creta,Diesel,"25,000 kms",1582,Manual,19.67 kmpl,5,-,126.2,Disc,Drum,400-litres,1630,1780,4270,-,-,190 Kmph,https://www.cardekho.com/used-cars+in+lucknow,10.50 Lakh


In [13]:
# Saving the scraped data into excel file
df.to_excel("Used_Cars.xlsx")

In [15]:
driver.close()  # to close the driver at the end

Finally we have scraped old cars data for important features from the mentioned website.